<a href="https://colab.research.google.com/github/lovellbrian/ELEC4630/blob/master/Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chollet P98

SIMPLE HOLD-OUT VALIDATION
Set apart some fraction of your data as your test set. Train on the remaining data, and
evaluate on the test set. As you saw in the previous sections, in order to prevent information
leaks, you shouldn’t tune your model based on the test set, and therefore you
should also reserve a validation set.
Schematically, hold-out validation looks like figure 4.1. The following listing shows
a simple implementation.

In [ ]:
num_validation_samples = 10000
np.random.shuffle(data)
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]
training_data = data[:]
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)
# At this point you can tune your model,
# retrain it, evaluate it, tune it again...
model = get_model()
model.train(np.concatenate([training_data,
validation_data]))
test_score = model.evaluate(test_data)

K-FOLD VALIDATION
With this approach, you split your data into K partitions of equal size. For each partition
i, train a model on the remaining K – 1 partitions, and evaluate it on partition i.
Your final score is then the averages of the K scores obtained. This method is helpful
when the performance of your model shows significant variance based on your traintest
split. Like hold-out validation, this method doesn’t exempt you from using a distinct
validation set for model calibration.
Schematically, K-fold cross-validation looks like figure 4.2. Listing 4.2 shows a simple
implementation.

In [ ]:
k = 4
num_validation_samples = len(data) // k
np.random.shuffle(data)
validation_scores = []
for fold in range(k):
  validation_data = data[num_validation_samples * fold:
  num_validation_samples * (fold + 1)]
  training_data = data[:num_validation_samples * fold] +
  data[num_validation_samples * (fold + 1):]
  model = get_model()
  model.train(training_data)
  validation_score = model.evaluate(validation_data)
  validation_scores.append(validation_score)
  validation_score = np.average(validation_scores)
  model = get_model()
  model.train(data)
  test_score = model.evaluate(test_data)

ITERATED K-FOLD VALIDATION WITH SHUFFLING
This one is for situations in which you have relatively little data available and you need
to evaluate your model as precisely as possible. I’ve found it to be extremely helpful in
Kaggle competitions. It consists of applying K-fold validation multiple times, shuffling
the data every time before splitting it K ways. The final score is the average of the
scores obtained at each run of K-fold validation. Note that you end up training and
evaluating P × K models (where P is the number of iterations you use), which can very
expensive.